In [1]:
from sailthru.sailthru_client import SailthruClient
import vertica_python
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from vertica_python import connect
import StringIO
from StringIO import StringIO
from datetime import date, timedelta as td
import xlsxwriter
import os
import json
import urllib2
import base64
import datetime
import re
import csv
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from IPython.display import display
import requests
import argparse
from dateutil import parser
from datetime import datetime, timedelta
from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [2]:
def get_service(api_name, api_version, scope, client_secrets_path):
  """Get a service that communicates to a Google API.

  Args:
    api_name: string The name of the api to connect to.
    api_version: string The api version to connect to.
    scope: A list of strings representing the auth scopes to authorize for the
      connection.
    client_secrets_path: string A path to a valid client secrets file.

  Returns:
    A service that is connected to the specified API.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      client_secrets_path, scope=scope,
      message=tools.message_if_missing(client_secrets_path))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage(api_name + '.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  service = build(api_name, api_version, http=http)

  return service

In [3]:
def get_ga_data(service, config_dict):
    
    df = pd.DataFrame()
    
    for i in pd.date_range(start_date, end_date):
        
        out = service.data().ga().get(
            ids='ga:' + '86300562',
            start_date=str(i.date()),
            end_date=str(i.date()),
            max_results=10000,
            metrics=','.join(config_dict['metric_list']),
            dimensions=','.join(config_dict['dimension_list']),
            filters=';'.join(config_dict['filter_list'])).execute()

        col_list = []

        for i in np.arange(0,len(out['columnHeaders'])):
            line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
            col_list.append(line)
        try:
            df = df.append(pd.DataFrame(out['rows'],columns=col_list))
        except:
            len(out) == 0
            
    return df


In [4]:
def extract_and_clean_df(config_dict, metric):
    df = get_ga_data(service, config_dict)
    
    if (metric != 'transactions'):
        df = df.rename(columns={'uniqueEvents': metric})
        df[metric] = pd.to_numeric(df[metric])
    else:
        df = df.rename(columns={'itemRevenue':'total_revenue',
                                'uniquePurchases':'transactions',
                                'productName':'eventLabel'})
        df['total_revenue'] = pd.to_numeric(df['total_revenue'])
        df['transactions'] = pd.to_numeric(df['transactions'])
    return df

In [5]:
def massage_df(df, metric):
    
    max_df = df.groupby('campaign').max()
    
    if (metric != 'transactions'):
        df_agg = df.reset_index().merge(df.groupby('campaign').sum().reset_index(), on='campaign',how='inner')
        df_agg = df_agg.rename(columns={'eventLabel_x':'top_%s_course' % metric,
                                        '%s_x' % metric:'%s_in_top_course' % metric,
                                        '%s_y' % metric:'%s_from_campaign' % metric                                                     
                                       })
        df_agg = df_agg.drop(['eventLabel_y'],axis=1)
    else:
        df_agg = df.merge(max_df.reset_index(), how='inner', on=['campaign', 'total_revenue', 'transactions']).groupby('campaign').first()
        df_agg = df_agg.reset_index().merge(df.groupby('campaign').sum().reset_index(), on='campaign',how='inner')
        df_agg = df_agg.rename(columns={'productName_x':'top_%s_course' % metric,
                                'total_revenue_x':'total_revenue_from_top_course',  
                                '%s_x' % metric:'%s_in_top_course' % metric,
                                'total_revenue_y':'total_revenue_from_campaign',  
                                '%s_y' % metric:'%s_from_campaign' % metric                                                     
                               })
        df_agg = df_agg.drop(['productName_y'],axis=1)
    return df_agg

In [6]:
def extract_payload(row):
    
    try:
        emails_sent = row['stats']['total']['count']
    except:
        emails_sent = 0
    
    try:
        unsubscribed = row['stats']['total']['optout']
    except:
        unsubscribed = 0
        
    try:
        unique_opens = row['stats']['total']['open_total']
    except:
        unique_opens = 0
        
    try:
        unique_sub_clicks = row['stats']['total']['click_total']
    except:
        unique_sub_clicks = 0
        
    dash_list = [pos for pos, char in enumerate(row['name']) if char == '_']
    
    if (len(dash_list) > 1):
        row['name'] = row['name'][0:dash_list[0]] + row['name'][dash_list[1]::]

    return row.append(pd.Series([emails_sent, unsubscribed, unique_opens, unique_sub_clicks]))

In [7]:
def clean_sailthru(df):
    df['campaign'] = df.campaign.str.lower()
    df['campaign'] = df.campaign.str.replace('_','-')
    df = df[df.campaign.str.contains('studentnewsletter')]
    df['unsub_rate'] = df['unsubscribed']*100.0/df['emails_sent']
    df['open_rate'] = df['unique_opens']*100.0/df['emails_sent']
    df['click_rate'] = df['unique_sub_clicks']*100.0/df['emails_sent']
    return df

In [8]:
def ga_timestamp(row):
    
    if (str(row['course_id'])=='nan'):
        row['date_diff'] = 0
    else:
        row['date'] = parser.parse(row['date'])
        row['date_diff'] = (row['date'] - row['7d_campaign_end']).days
    return row

def sailthru_timestamp(row, date_col='start_time'):
    row[date_col] = parser.parse(row[date_col])
    return row

In [9]:
def add_metrics(df):
    df['registrations_conversion_rate'] = df['registrations']*100.0/df['emails_sent']
    df['enrollments_conversion_rate'] = df['enrollments']*100.0/df['emails_sent']
    df['dollars_per_email'] = df['total_revenue']*100.0/df['emails_sent']
    return df

In [10]:
scope = ['https://www.googleapis.com/auth/analytics.readonly']
service = get_service('analytics', 'v3', scope, 'client_secrets.json')

In [11]:
start_date = '2016-10-15'
end_date = '2016-11-01'

In [13]:
start='2015-09-20T00:00:00+00:00'
end='2015-12-21T00:00:00+00:00'

In [41]:
r = requests.auth.HTTPBasicAuth('x', '7b87ccd203b973d87d0ac4423192afa6-us5')
total_items = requests.get("http://us5.api.mailchimp.com/3.0/reports/?fields=total_items&since_send_time="+str(start) +"&before_send_time="+str(end), auth=r)
(json.loads(total_items.content))


# In[13]:

#r = requests.auth.HTTPBasicAuth('x', 'd034431d4d586f7c81d56405235f6131-us5')
#total_items = requests.get("http://us5.api.mailchimp.com/3.0/reports/?fields=total_items&since_send_time="+str(start) +"&before_send_time="+str(end), auth=r)
#n = (json.loads(total_items.content)['total_items'])

#print n

# t=0
# for x in range(0, n):
#     try:
#         data = requests.get("http://us5.api.mailchimp.com/3.0/reports/?count=1&offset="+str(x)+"&fields=reports&since_send_time="+str(start) +"&before_send_time="+str(end), auth=r)
#         data2 = json.loads(data.content)
#         data3=DataFrame(data2)
#         t=t+1
#         if t==1:
#             mcData=data3
#         if t>1:
#             mcData=pd.concat([mcData,data3])
#     except:
#         pass
# print "DONE"

{u'total_items': 69}

In [42]:
total_items.content

'{"total_items":69}'

In [43]:
t=0
for x in range(0, 69):
    try:
        data = requests.get("http://us5.api.mailchimp.com/3.0/reports/?count=1&offset="+str(x)+"&fields=reports&since_send_time="+str(start) +"&before_send_time="+str(end), auth=r)
        data2 = json.loads(data.content)
        data3=DataFrame(data2)
        t=t+1
        if t==1:
            mcData=data3
        if t>1:
            mcData=pd.concat([mcData,data3])
    except:
        pass
print "DONE"

DONE


In [69]:
from pandas.io.json import json_normalize

In [72]:
json.loads(data.content)

{u'reports': [{u'_links': [{u'href': u'https://us5.api.mailchimp.com/3.0/reports',
     u'method': u'GET',
     u'rel': u'parent',
     u'schema': u'https://us5.api.mailchimp.com/schema/3.0/CollectionLinks/Reports.json',
     u'targetSchema': u'https://us5.api.mailchimp.com/schema/3.0/Definitions/Reports/CollectionResponse.json'},
    {u'href': u'https://us5.api.mailchimp.com/3.0/reports/9a9c09c106',
     u'method': u'GET',
     u'rel': u'self',
     u'targetSchema': u'https://us5.api.mailchimp.com/schema/3.0/Definitions/Reports/Response.json'},
    {u'href': u'https://us5.api.mailchimp.com/3.0/campaigns/9a9c09c106',
     u'method': u'GET',
     u'rel': u'campaign',
     u'targetSchema': u'https://us5.api.mailchimp.com/schema/3.0/Definitions/Campaigns/Response.json'},
    {u'href': u'https://us5.api.mailchimp.com/3.0/reports/9a9c09c106/sub-reports',
     u'method': u'GET',
     u'rel': u'sub-reports',
     u'targetSchema': u'https://us5.api.mailchimp.com/schema/3.0/Definitions/Reports/

In [65]:
pd.DataFrame(json.dumps(json.loads(data.content))

PandasError: DataFrame constructor not properly called!

In [53]:
pd.DataFrame(json.loads(data.content))

,reports
0,"{u'forwards': {u'forwards_count': 0, u'forward..."


In [61]:
type(json.decode(data.content))

AttributeError: 'module' object has no attribute 'decode'

In [629]:
enrollments_dict = {
    'filter_list': [
'ga:eventAction==edx.course.enrollment.activated',
'ga:campaign=~newsletter',
'ga:medium==email',
'ga:source==sailthru'

],
    'dimension_list': [
'ga:date',
'ga:campaign',
'ga:eventLabel'
],
    'metric_list': [
'ga:uniqueEvents'
],
}

In [630]:
registrations_dict = {
    'filter_list': [
'ga:eventAction==edx.bi.user.account.registered',
'ga:campaign=~newsletter',
'ga:medium==email',
'ga:source==sailthru'

],
    'dimension_list': [
'ga:date',
'ga:campaign',
'ga:eventLabel'
],
    'metric_list': [
'ga:uniqueEvents'
],
}

In [631]:
transactions_dict = {
    'filter_list': [
'ga:campaign=~newsletter',
'ga:medium==email',
'ga:source==sailthru'

],
    'dimension_list': [
'ga:date',
'ga:campaign',
'ga:productName',
],
    'metric_list': [
'ga:itemRevenue',
'ga:uniquePurchases'
],
}

In [632]:
ga_registrations = extract_and_clean_df(registrations_dict, 'registrations')
ga_enrollments = extract_and_clean_df(enrollments_dict, 'enrollments')
ga_transactions = extract_and_clean_df(transactions_dict, 'transactions')

In [719]:
#combine all ga data into one df

ga_master = (ga_registrations.merge(
        ga_enrollments, how='outer', on = ['campaign','eventLabel','date'])).merge(
    ga_transactions, how='outer', on = ['campaign','eventLabel','date']).reset_index()
ga_master = ga_master.drop(['index'],axis=1)
ga_master = ga_master.rename(columns={'eventLabel':'course_id'})

In [73]:
api_key = 'd50d1dac0ae6f50b885a618a7d36d4e0'
api_secret = '844bf4965e85e43da2267b8ac0fb5c1c'
sc = sailthru_client = SailthruClient(api_key, api_secret)

request_data = {'status': 'sent', 'start_date':'2016-01-01'}
response = sc.api_get('blast', request_data).get_body()

sailthru_campaign_stats = DataFrame(response['blasts'])[['name','blast_id','start_time','stats']]

In [74]:
#extract data, rename columns

sailthru_campaign_stats = sailthru_campaign_stats.apply(extract_payload, axis=1)
sailthru_campaign_stats = sailthru_campaign_stats.rename(columns={'name':'campaign',
                    0:'emails_sent',
                   1:'unsubscribed',
                   2:'unique_opens',
                   3:'unique_sub_clicks'})
sailthru_campaign_stats = sailthru_campaign_stats.apply(sailthru_timestamp, axis=1)
sailthru_campaign_stats['7d_campaign_end'] = sailthru_campaign_stats.start_time + timedelta(days=7) 

In [82]:
sailthru_campaign_stats.to_csv(path_or_buf = '../script_output/bisupport-29.csv') 

In [723]:
#de-dupe table to map one sailthru campaign to a 7d end date

sailthru_campaign_7d = sailthru_campaign_stats.groupby('campaign').first()['7d_campaign_end'].reset_index()
sailthru_campaign_7d['campaign'] = sailthru_campaign_7d.campaign.str.lower().str.replace('_','-')
sailthru_campaign_7d = sailthru_campaign_7d[sailthru_campaign_7d.campaign.str.contains('studentnewsletter')]

In [724]:
sailthru_campaign_stats_summarized = (sailthru_campaign_stats.groupby('campaign').sum()).reset_index().drop(['blast_id'], axis=1)
sailthru_campaign_stats_summarized = clean_sailthru(sailthru_campaign_stats_summarized)
sailthru_campaign_stats_summarized = sailthru_campaign_stats_summarized.merge(sailthru_campaign_7d, how='inner', on='campaign')

/Users/adleyhemphill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/adleyhemphill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/adleyhemphill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [748]:
def df_agg(df, agg_level):
    
    if (agg_level == 'course'):
        df_all = df.groupby(['campaign','course_id']).sum().reset_index()
        df_7d = (df[df.date_diff<=7].groupby(['campaign','course_id']).sum().reset_index())
        df_out = df_all.merge(df_7d, how='left', on=['campaign','course_id'], suffixes=('','_7d'))
        return df_out#.drop(['date_diff','date_diff_7d'],axis=1)
    elif (agg_level == 'campaign'):
        df_all = df.groupby(['campaign']).sum().reset_index()
        df_7d = (df[df.date_diff<=7].groupby(['campaign']).sum().reset_index())
        df_out =  df_all.merge(df_7d, how='left', on=['campaign'], suffixes=('','_7d'))
        return df_out#.drop(['date_diff','date_diff_7d'],axis=1)

In [749]:
#combine all ga and sailthru data and clean up the date info so we can do the 7d calculations

ga_sailthru_combined = sailthru_campaign_stats_summarized[['campaign','7d_campaign_end']].merge(ga_master, how='left', on='campaign')
ga_sailthru_combined = ga_sailthru_combined.apply(ga_timestamp, axis=1)

In [753]:
#aggregate at the campaign and course level

ga_sailthru_combined_campaign = df_agg(ga_sailthru_combined, 'campaign')
ga_sailthru_combined_course = df_agg(ga_sailthru_combined, 'course')

In [755]:
#join back to get the sailthru stats

ga_sailthru_combined_campaign = sailthru_campaign_stats_summarized.merge(
    ga_sailthru_combined_campaign, how='left', on='campaign').drop(['date_diff','date_diff_7d','7d_campaign_end'],axis=1)
ga_sailthru_combined_course = sailthru_campaign_stats_summarized.merge(
    ga_sailthru_combined_course, how='left', on='campaign').drop(['date_diff','date_diff_7d','7d_campaign_end'],axis=1)

In [756]:
#add conversion rate metrics

ga_sailthru_combined_campaign = add_metrics(ga_sailthru_combined_campaign)
ga_sailthru_combined_course = add_metrics(ga_sailthru_combined_course)

In [757]:
ga_sailthru_combined_campaign

,campaign,emails_sent,unsubscribed,unique_opens,unique_sub_clicks,unsub_rate,open_rate,click_rate,registrations,enrollments,total_revenue,transactions,registrations_7d,enrollments_7d,total_revenue_7d,transactions_7d,registrations_conversion_rate,enrollments_conversion_rate,dollars_per_email
0,spanishstudentnewsletter-20160906,441301,416,107737,13265,0.094267,24.413496,3.005885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,studentnewsletter-20161004,5250654,8390,1042368,107321,0.159790,19.852156,2.043955,361.0,9726.0,15227.9,245.0,147.0,4358.0,7118.8,107.0,0.006875,0.185234,0.290019
2,studentnewsletter-20161011,5283703,7985,1035597,123166,0.151125,19.599834,2.331055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,studentnewsletter-20161018,5320519,8377,1086659,103052,0.157447,20.423929,1.936879,871.0,26962.0,19115.7,309.0,871.0,26962.0,19115.7,309.0,0.016371,0.506755,0.359283
4,studentnewsletter-20161025,4297053,6286,856874,98199,0.146286,19.940969,2.285264,733.0,27572.0,18617.7,290.0,733.0,27572.0,18617.7,290.0,0.017058,0.641649,0.433267
5,studentnewsletter-20161101,5402604,8327,1027413,114807,0.154129,19.016996,2.125031,337.0,12559.0,4134.0,73.0,337.0,12559.0,4134.0,73.0,0.006238,0.232462,0.076519
6,studentnewsletter-20161108,5435475,7246,1031816,88374,0.133309,18.982996,1.625874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,studentnewsletter-20161115,5461811,7699,1096963,119264,0.140961,20.084236,2.183598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,studentnewsletter-20161122,5502433,8149,1136395,110891,0.148098,20.652591,2.015309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,studentnewsletter-20161129,5578288,9653,1081749,93617,0.173046,19.392132,1.678239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [587]:
#master_ga_sailthru_df = master_ga_sailthru_df.replace(to_replace='event', value='homepage')

In [763]:
excelFile='./../script_output/newsletter_sample2.xlsx'
writer = pd.ExcelWriter(excelFile,engine='xlsxwriter')

ga_sailthru_combined_campaign.to_excel(writer, index=False, sheet_name='campaign_summary', startrow=2)
ga_sailthru_combined_course.to_excel(writer, index=False, sheet_name='course_summary', startrow=2)
ga_sailthru_combined_campaign[ga_sailthru_combined_campaign.campaign.str.contains('^studentnewsletter')].to_excel(writer, index=False, sheet_name='tuesday_campaign_summary', startrow=2)
ga_sailthru_combined_course[ga_sailthru_combined_course.campaign.str.contains('^studentnewsletter')].to_excel(writer, index=False, sheet_name='tuesday_course_summary', startrow=2)

# Get access to the workbook and sheet
workbook = writer.book
worksheet = writer.sheets['campaign_summary']

money_fmt = workbook.add_format({'num_format': '$#,##0', 'bold': True})
money_fmt_long = workbook.add_format({'num_format': '$#,##0.00000', 'bold': True})
percent_fmt = workbook.add_format({'num_format': '0.0000%', 'bold': False})
comma_fmt_long = workbook.add_format({'num_format': '#,##0.000', 'bold': False})
comma_fmt_short = workbook.add_format({'num_format': '#,##0', 'bold': False})
date_fmt = workbook.add_format({'num_format': 'dd/mm/yy'})
cell_format = workbook.add_format({'bold': True, 'italic': False})

worksheet.conditional_format('Q1:Q10000', {'type': '3_color_scale'})
worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:H', 15, comma_fmt_long)
worksheet.set_column('I:J', 20, comma_fmt_short)
worksheet.set_column('K:K', 20, money_fmt)
worksheet.set_column('L:N', 20, comma_fmt_short)
worksheet.set_column('O:O', 20, money_fmt)
worksheet.set_column('P:P', 20, comma_fmt_short)
worksheet.set_column('Q:R', 20, comma_fmt_long)
worksheet.set_column('S:S', 20, money_fmt_long)

#worksheet.write('A1', 'Newsletters with over 1M sends, Data from '+str(report_d1)+' to '+str(d2) , cell_format)

worksheet = writer.sheets['course_summary']

worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})
worksheet.conditional_format('T1:T10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:I', 15, comma_fmt_long)
worksheet.set_column('J:K', 20, comma_fmt_short)
worksheet.set_column('L:L', 20, money_fmt)
worksheet.set_column('M:O', 20, comma_fmt_short)
worksheet.set_column('P:P', 20, money_fmt)
worksheet.set_column('Q:Q', 20, comma_fmt_short)
worksheet.set_column('R:S', 20, comma_fmt_long)
worksheet.set_column('T:T', 20, money_fmt_long)


worksheet = writer.sheets['tuesday_campaign_summary']

worksheet.conditional_format('Q1:Q10000', {'type': '3_color_scale'})
worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:H', 15, comma_fmt_long)
worksheet.set_column('I:J', 20, comma_fmt_short)
worksheet.set_column('K:K', 20, money_fmt)
worksheet.set_column('L:N', 20, comma_fmt_short)
worksheet.set_column('O:O', 20, money_fmt)
worksheet.set_column('P:P', 20, comma_fmt_short)
worksheet.set_column('Q:R', 20, comma_fmt_long)
worksheet.set_column('S:S', 20, money_fmt_long)

worksheet = writer.sheets['tuesday_course_summary']

worksheet.conditional_format('R1:R10000', {'type': '3_color_scale'})
worksheet.conditional_format('S1:S10000', {'type': '3_color_scale'})
worksheet.conditional_format('T1:T10000', {'type': '3_color_scale'})

worksheet.set_column('A:A', 15, date_fmt)
worksheet.set_column('B:I', 15, comma_fmt_long)
worksheet.set_column('J:K', 20, comma_fmt_short)
worksheet.set_column('L:L', 20, money_fmt)
worksheet.set_column('M:O', 20, comma_fmt_short)
worksheet.set_column('P:P', 20, money_fmt)
worksheet.set_column('Q:Q', 20, comma_fmt_short)
worksheet.set_column('R:S', 20, comma_fmt_long)
worksheet.set_column('T:T', 20, money_fmt_long)

writer.save()


#WHEN ONLY GOOGLE DATA IS WORKING

# summary['dateSent'] = summary['dateSent'].dt.date
# courseDetail['dateSent'] = courseDetail['dateSent'].dt.date

# writer = pd.ExcelWriter(str(filepath)+'NL Data Missing MailChimp Data '+str(d1)+' to '+str(d2)+'.xlsx',engine='xlsxwriter')
# summary.to_excel(writer,sheet_name='CampaignEnrolls',index=False, startrow=2)
# courseDetail.to_excel(writer,sheet_name='courseEnrolls',index=False, startrow=2)

# # Get access to the workbook and sheet
# workbook = writer.book

# worksheet = writer.sheets['CampaignEnrolls']

# money_fmt = workbook.add_format({'num_format': '$#,##0', 'bold': True})
# percent_fmt = workbook.add_format({'num_format': '0.0%', 'bold': False})
# comma_fmt = workbook.add_format({'num_format': '#,##0', 'bold': False})
# date_fmt = workbook.add_format({'num_format': 'dd/mm/yy'})
# cell_format = workbook.add_format({'bold': True, 'italic': False})

# worksheet.set_column('A:A', 55)
# worksheet.set_column('B:B', 15)
# worksheet.set_column('C:D', 20, comma_fmt)
# worksheet.set_column('E:E', 15)
# worksheet.write('A1', 'Newsletter GA Campaign Enrolls, Data from '+str(d1)+' to '+str(d2) , cell_format)

# worksheet = writer.sheets['courseEnrolls']
# worksheet.set_column('A:A', 44)
# worksheet.set_column('B:B', 15)
# worksheet.set_column('C:D', 20, comma_fmt)
# worksheet.set_column('E:E', 55)
# worksheet.set_column('F:G', 20, comma_fmt)
# worksheet.write('A1', 'Newsletter GA Course Enrolls, Data from '+str(d1)+' to '+str(d2) , cell_format)
# worksheet.autofilter('A3:G3')


# writer.save()
